<h2>Google Sector Report</h2>
<h4>This function crawls the google finance webpage and returns:</h4>
<br>1. The sector name
<br>2. The percentage change in that sector
<br>3. The biggest gainer and its percentage change of that sector
<br>4. The biggest loser and its percentage change of that sector

In [1]:
def google_sector_report():
    import requests
    from bs4 import BeautifulSoup
    url='http://www.google.com/finance'
    response=requests.get(url)
    status='GOOD'
    dic={}
    
    if response.status_code==200:
        data_soup=BeautifulSoup(response.content,'lxml')
        sector_list=data_soup.find_all('div',{'class':"id-secperf sfe-section-major"})
        for tag in sector_list:
            name_list=tag.find_all('a')
            change_list=tag.find_all('span')

        i=0  
        for element in name_list:
            url2='http://www.google.com'+element.get('href')
            response=requests.get(url2)
            if response.status_code!=200:
                status='BAD'
                break
            data_soup2=BeautifulSoup(response.content,'lxml')
            biggest_list=data_soup2.find('div', {'class':"sfe-break-bottom"})
            tr_list=biggest_list.find_all('tr')

            dic[element.get_text()]={}
            dic[element.get_text()]['change']=change_list[i].get_text()
            i=i+1

            dic[element.get_text()]['biggest_gainer']={}
            dic[element.get_text()]['biggest_loser']={}
            
            b_list=biggest_list.find_all('b')
            b_list=[item.get_text() for item in b_list]
            b_str=','.join(b_list)

            if 'Gainers' in b_str:
                dic[element.get_text()]['biggest_gainer']['change']=tr_list[1]('span')[1].get_text()[1:-1]
                dic[element.get_text()]['biggest_gainer']['equity']=tr_list[1]('a')[0].get_text()
                if 'Losers' in b_str:
                    dic[element.get_text()]['biggest_loser']['change']=tr_list[7]('span')[1].get_text()[1:-1]
                    dic[element.get_text()]['biggest_loser']['equity']=tr_list[7]('a')[0].get_text()
            elif 'Gainers' not in b_str:
                dic[element.get_text()]['biggest_gainer']['change']=None
                dic[element.get_text()]['biggest_gainer']['equity']=""
                if 'Losers' in b_str:
                    dic[element.get_text()]['biggest_loser']['change']=tr_list[1]('span')[1].get_text()[1:-1]
                    dic[element.get_text()]['biggest_loser']['equity']=tr_list[1]('a')[0].get_text()
            if 'Losers' not in b_str:
                dic[element.get_text()]['biggest_loser']['change']=None
                dic[element.get_text()]['biggest_loser']['equity']=""
                
        import json
        json_dic=json.dumps({'status':status,'result':dic}, sort_keys=True)
        return json.loads(json_dic)
    else:
        status='BAD'
        import json
        json_dic=json.dumps({'status':status,'result':dic}, sort_keys=True)
        return json.loads(json.dic)

<h3>Let's run the function and see how it looks like</h3>

In [2]:
google_sector_report()

{'result': {'Basic Materials': {'biggest_gainer': {'change': '13.19%',
    'equity': 'FMC Corp'},
   'biggest_loser': {'change': '-4.67%', 'equity': 'Synalloy Corporation'},
   'change': '-0.07%'},
  'Cyclical Cons. Goods ...': {'biggest_gainer': {'change': '10.08%',
    'equity': 'Commercial Vehicle Group'},
   'biggest_loser': {'change': '-6.83%', 'equity': 'Beasley Broadcast Group'},
   'change': '-0.16%'},
  'Energy': {'biggest_gainer': {'change': '8.53%',
    'equity': 'Legacy Reserves LP'},
   'biggest_loser': {'change': '-6.83%', 'equity': 'PHI Inc.'},
   'change': '-0.04%'},
  'Financials': {'biggest_gainer': {'change': '22.79%',
    'equity': 'Fang Holdings Ltd'},
   'biggest_loser': {'change': '-7.63%', 'equity': 'Oak Valley Bancorp'},
   'change': '-0.37%'},
  'Healthcare': {'biggest_gainer': {'change': '10.64%',
    'equity': 'Sangamo Therapeutics Inc'},
   'biggest_loser': {'change': '-21.50%', 'equity': 'Acorda Therapeutics Inc'},
   'change': '-0.21%'},
  'Industrials': 